In [1]:
# "Sanaz Hosseini"- University of North Carolina at Charlotte
# Introduction to Machine Learning Class - Instructor: Prof. Hamed Tabkhi
# Fully Connected Neural Networks VS Convolutional Neural Network
# Homework 6

In [23]:
import torch
import time
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
%matplotlib inline

In [24]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Files already downloaded and verified


In [25]:
torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [26]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [27]:
import torch.nn as nn

n_out = 10

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, n_out),
        )

In [28]:
loss = nn.NLLLoss()

In [29]:
img, label = cifar10[0]

out = model(img.view(-1).unsqueeze(0))

loss(out, torch.tensor([label]))

tensor(0.1085, grad_fn=<NllLossBackward0>)

In [30]:
import datetime  
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.LogSoftmax(dim=1))

model.to(device)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 300

for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        training_start_time = time.time()
        for imgs, labels in train_loader:  
            imgs=imgs.to(device)
            labels=labels.to(device)
            outputs = model(imgs.view(imgs.shape[0], -1))
            loss = loss_fn(outputs, labels) 
            
            optimizer.zero_grad() 
            loss.backward()  
            optimizer.step()  
            loss_train += loss.item() 
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  

2022-12-10 18:36:07.859248 Epoch 1, Training loss 1.673237381383891
2022-12-10 18:38:56.135501 Epoch 10, Training loss 0.9060790273539551
2022-12-10 18:42:07.764159 Epoch 20, Training loss 0.541906619030039
2022-12-10 18:45:14.249325 Epoch 30, Training loss 0.37347608223519363
2022-12-10 18:48:23.054310 Epoch 40, Training loss 0.2783795592882444
2022-12-10 18:51:30.359507 Epoch 50, Training loss 0.249001441918828
2022-12-10 18:54:36.339435 Epoch 60, Training loss 0.20147149199548434
2022-12-10 18:57:43.594684 Epoch 70, Training loss 0.18866344788076017
2022-12-10 19:00:51.242258 Epoch 80, Training loss 0.16184990525202791
2022-12-10 19:03:58.034996 Epoch 90, Training loss 0.14581574769774
2022-12-10 19:07:08.279758 Epoch 100, Training loss 0.14347260941982345
2022-12-10 19:10:27.905223 Epoch 110, Training loss 0.13713459560797667
2022-12-10 19:13:37.920145 Epoch 120, Training loss 0.11866026777473976
2022-12-10 19:16:45.750114 Epoch 130, Training loss 0.1083180743809718
2022-12-10 19:1

In [31]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs=imgs.to(device)
        labels=labels.to(device)
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
       
print("Accuracy: %f" % (correct / total))

Accuracy: 0.986940


In [32]:
import datetime  
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)
model = nn.Sequential(
            nn.Linear(3072, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 10),
            nn.LogSoftmax(dim=1))

model.to(device)
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.NLLLoss()
n_epochs = 300

for epoch in range(1, n_epochs + 1): 
        loss_train = 0.0
        training_start_time = time.time()
        for imgs, labels in train_loader:  
            imgs=imgs.to(device)
            labels=labels.to(device)
            outputs = model(imgs.view(imgs.shape[0], -1))
            loss = loss_fn(outputs, labels) 
            
            optimizer.zero_grad() 
            loss.backward()  
            optimizer.step()  
            loss_train += loss.item() 
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  

2022-12-10 20:23:26.505723 Epoch 1, Training loss 1.6803107173241618
2022-12-10 20:25:06.312002 Epoch 10, Training loss 0.8693386975990232
2022-12-10 20:26:57.031874 Epoch 20, Training loss 0.4812492913640369
2022-12-10 20:28:48.256844 Epoch 30, Training loss 0.330466573248091
2022-12-10 20:30:38.902300 Epoch 40, Training loss 0.2644229508993571
2022-12-10 20:32:29.602312 Epoch 50, Training loss 0.21903069929727126
2022-12-10 20:34:20.904219 Epoch 60, Training loss 0.18474198846846743
2022-12-10 20:36:12.032271 Epoch 70, Training loss 0.1722965449454439
2022-12-10 20:38:03.206550 Epoch 80, Training loss 0.14638836496471502
2022-12-10 20:39:54.957567 Epoch 90, Training loss 0.155758059071496
2022-12-10 20:41:46.504041 Epoch 100, Training loss 0.1280352481250244
2022-12-10 20:43:41.212320 Epoch 110, Training loss 0.13180096306816658
2022-12-10 20:45:35.642286 Epoch 120, Training loss 0.12571092947627968
2022-12-10 20:47:28.342278 Epoch 130, Training loss 0.13017092507434985
2022-12-10 20

In [33]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in train_loader:
        imgs=imgs.to(device)
        labels=labels.to(device)
        outputs = model(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
       
print("Accuracy: %f" % (correct / total))

Accuracy: 0.992920
